In [3]:
!pip install opencv-python

In [3]:
import cv2


In [1]:
!pip install opencv-python==4.5.5.64

In [5]:
print(cv2.__version__)

4.5.5


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np

**IMPORTING WITH THE HELP OF API**

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.45G/1.45G [00:20<00:00, 75.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1


In [9]:
train_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train"
valid_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid"
test_dir =r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test"

#set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


***BUILDING THE CNN MODEL***

In [13]:
model = Sequential([
    Conv2D(32,(3,3),activation ='relu',input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64,(3,3),activation ='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

**TRAINING THE MODEL**

In [20]:
history=model.fit(train_generator,epochs=3,validation_data=valid_generator,verbose=1)

Epoch 1/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 236s 249ms/step - accuracy: 0.9781 - loss: 0.0570 - val_accuracy: 0.9503 - val_loss: 0.1708
Epoch 2/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 256s 243ms/step - accuracy: 0.9761 - loss: 0.0628 - val_accuracy: 0.9632 - val_loss: 0.1212
Epoch 3/3
946/946 ━━━━━━━━━━━━━━━━━━━━ 227s 240ms/step - accuracy: 0.9828 - loss: 0.0481 - val_accuracy: 0.9486 - val_loss: 0.2107


In [22]:
model.save('wildfire_model.h5')

**building GUI with Tkinter**

In [24]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

model=load_model('wildfire_model.h5')
print("Model loaded successfully")

Model loaded successfully


In [25]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [27]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('wildfire_model.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated input definition
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains a wildfire."
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f9a3f62f66b7ebfda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
